In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import RobustScaler

In [3]:
file_path = 'prepped_data.pkl'
df = pd.read_pickle(file_path)

print("✅ Data loaded successfully.")
print(df.info(verbose=True, max_cols=30))

✅ Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
Index: 2013945 entries, 0 to 17284449
Data columns (total 22 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Unnamed: 0       int64         
 1   ssn              string        
 2   cc_num           int64         
 3   city             string        
 4   zip              int64         
 5   lat              float64       
 6   long             float64       
 7   city_pop         int64         
 8   job              string        
 9   dob              datetime64[ns]
 10  acct_num         int64         
 11  profile          string        
 12  trans_num        string        
 13  trans_date       datetime64[ns]
 14  category         string        
 15  amt              float64       
 16  is_fraud         int64         
 17  merchant         string        
 18  merch_lat        float64       
 19  merch_long       float64       
 20  is_male          int64         
 21  trans_t

In [5]:
null_counts = df.isnull().sum()
print(null_counts.to_markdown())

|                 |   0 |
|:----------------|----:|
| Unnamed: 0      |   0 |
| ssn             |   0 |
| cc_num          |   0 |
| city            |   0 |
| zip             |   0 |
| lat             |   0 |
| long            |   0 |
| city_pop        |   0 |
| job             |   0 |
| dob             |   0 |
| acct_num        |   0 |
| profile         |   0 |
| trans_num       |   0 |
| trans_date      |   0 |
| category        |   0 |
| amt             |   0 |
| is_fraud        |   0 |
| merchant        |   0 |
| merch_lat       |   0 |
| merch_long      |   0 |
| is_male         |   0 |
| trans_timestamp |   0 |


In [6]:
df.describe()

,Unnamed: 0,cc_num,zip,lat,long,city_pop,dob,acct_num,trans_date,amt,is_fraud,merch_lat,merch_long,is_male,trans_timestamp
count,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2013945,2.013945e+06,2013945,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2.013945e+06,2013945
mean,1.708518e+07,3.279698e+17,9.294059e+04,3.542451e+01,-1.193474e+02,3.484885e+05,1979-01-03 02:52:46.610011808,5.029883e+11,2020-07-20 00:50:06.493225728,6.622737e+01,5.409284e-03,3.542440e+01,-1.193471e+02,4.900392e-01,2020-07-20 18:03:34.725393664
min,0.000000e+00,6.040566e+10,9.000100e+04,3.256260e+01,-1.242514e+02,4.400000e+01,1928-01-10 00:00:00,2.981017e+08,2020-01-01 00:00:00,1.000000e+00,0.000000e+00,3.156713e+01,-1.252498e+02,0.000000e+00,2020-01-01 00:01:13
25%,8.353373e+06,3.606912e+13,9.173000e+04,3.389980e+01,-1.214345e+02,5.118000e+04,1968-01-30 00:00:00,2.503429e+11,2020-04-25 00:00:00,8.170000e+00,0.000000e+00,3.368678e+01,-1.213295e+02,0.000000e+00,2020-04-25 23:17:43
50%,1.704962e+07,2.720745e+15,9.269200e+04,3.416900e+01,-1.183989e+02,1.036890e+05,1980-08-20 00:00:00,4.997494e+11,2020-07-22 00:00:00,3.286000e+01,0.000000e+00,3.463017e+01,-1.187317e+02,0.000000e+00,2020-07-22 13:45:33
75%,2.515087e+07,4.456813e+15,9.455800e+04,3.756050e+01,-1.178684e+02,2.398790e+05,1992-11-15 00:00:00,7.569859e+11,2020-10-21 00:00:00,7.666000e+01,0.000000e+00,3.743364e+01,-1.177016e+02,1.000000e+00,2020-10-21 21:50:27
max,3.462019e+07,4.997234e+18,9.615000e+04,4.178170e+01,-1.153775e+02,2.383912e+06,2008-11-01 00:00:00,9.993899e+11,2020-12-31 00:00:00,3.436294e+04,1.000000e+00,4.278163e+01,-1.143804e+02,1.000000e+00,2020-12-31 23:59:48
std,9.809598e+06,1.165179e+18,1.768583e+03,2.049900e+00,1.971902e+00,6.029027e+05,NaN,2.875887e+11,NaN,1.917003e+02,7.334866e-02,2.130184e+00,2.054473e+00,4.999009e-01,NaN


In [7]:
df.is_fraud.value_counts()

is_fraud
0    2003051
1      10894
Name: count, dtype: int64

In [8]:
df['is_fraud'].value_counts(normalize=True) * 100

is_fraud
0    99.459072
1     0.540928
Name: proportion, dtype: float64

In [9]:
df.shape

(2013945, 22)

In [ ]:
plt.figure(figsize=(8, 6))
sb.boxplot(x=df['amt'])
plt.title('Box Plot of Transaction Amount')
plt.savefig('amt_boxplot.png')
plt.clf()
plt.show()

<Figure size 800x600 with 0 Axes>

##Outlier Analasys using IQR
#IQR for imbalanced data considirations
#Analayze frauds rates within ouliers
#keep outlier flag for information keeping

In [11]:
FEATURES_TO_CHECK = ['city_pop', 'amt'] 
outlier_summary = []
baseline_fraud_rate = (df['is_fraud'].sum() / len(df)) * 100

# Initialize the combined outlier flag
df['IS_ANY_OUTLIER_IQR'] = False 

print("\n--- Outlier Detection and Analysis using IQR ---")

for col in FEATURES_TO_CHECK:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    UPPER_BOUND = Q3 + 1.5 * IQR
    LOWER_BOUND = Q1 - 1.5 * IQR
    
    outlier_mask = (df[col] > UPPER_BOUND) | (df[col] < LOWER_BOUND)
    
    df['IS_ANY_OUTLIER_IQR'] = df['IS_ANY_OUTLIER_IQR'] | outlier_mask
    
    df_outliers = df[outlier_mask].copy()
    total_outliers = len(df_outliers)
    fraud_outliers = df_outliers['is_fraud'].sum()
    
    # Calculate Fraud Rate within the Outliers
    fraud_percentage = (fraud_outliers / total_outliers) * 100 if total_outliers > 0 else 0
        
    outlier_summary.append({
        'Feature': col,
        'Total_Outliers_IQR': total_outliers,
        'Fraud_Outliers_IQR': fraud_outliers,
        'Fraud_in_Outliers_IQR (%)': fraud_percentage
    })
    
    print(f"\nFeature: {col}")
    print(f"  Total Outliers (IQR Method): {total_outliers:,}")
    print(f"  Fraud Rate in Outliers: {fraud_percentage:.4f}%")
    
#Final Reporting and Flag Verification ---
summary_df = pd.DataFrame(outlier_summary)
print("\nFinal Summary Table (IQR Outlier Analysis):")
print(f"Overall Baseline Fraud Rate: {baseline_fraud_rate:.4f}%")
print(summary_df.to_markdown(index=False, floatfmt='.4f'))

df['IS_ANY_OUTLIER_IQR'] = df['IS_ANY_OUTLIER_IQR'].astype(int)

total_any_outliers = df['IS_ANY_OUTLIER_IQR'].sum()
fraud_in_flagged = df[df['IS_ANY_OUTLIER_IQR'] == 1]['is_fraud'].mean() * 100

print(f"\n--- 'IS_ANY_OUTLIER_IQR' Flag Verification ---")
print(f"Total Records with Outlier Flag (1): {total_any_outliers:,}")
print(f"Fraud Rate for Flagged Records: {fraud_in_flagged:.4f}%")

#Capping Outliers using IQR bounds
for col in FEATURES_TO_CHECK:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    UPPER_BOUND = Q3 + 1.5 * IQR
    LOWER_BOUND = Q1 - 1.5 * IQR

    df[col] = np.where(df[col] > UPPER_BOUND, UPPER_BOUND, df[col])
    df[col] = np.where(df[col] < LOWER_BOUND, LOWER_BOUND, df[col])

print("\n✅ True Outliers capped using IQR bounds (1.5 * IQR).")





--- Outlier Detection and Analysis using IQR ---

Feature: city_pop
  Total Outliers (IQR Method): 341,480
  Fraud Rate in Outliers: 0.5951%

Feature: amt
  Total Outliers (IQR Method): 114,263
  Fraud Rate in Outliers: 7.3576%

Final Summary Table (IQR Outlier Analysis):
Overall Baseline Fraud Rate: 0.5409%
| Feature   |   Total_Outliers_IQR |   Fraud_Outliers_IQR |   Fraud_in_Outliers_IQR (%) |
|:----------|---------------------:|---------------------:|----------------------------:|
| city_pop  |               341480 |                 2032 |                      0.5951 |
| amt       |               114263 |                 8407 |                      7.3576 |

--- 'IS_ANY_OUTLIER_IQR' Flag Verification ---
Total Records with Outlier Flag (1): 436,774
Fraud Rate for Flagged Records: 2.0313%

✅ True Outliers capped using IQR bounds (1.5 * IQR).


In [18]:
final_cleaned_path = 'clensed_data.pkl'
df.to_pickle(final_cleaned_path)
print(f"\nFinal cleaned data saved to: {final_cleaned_path}")


Final cleaned data saved to: clensed_data.pkl
